# Claude Tools - Colab Processor
This notebook runs in Google Colab and processes requests from Claude Tools

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')
print("✅ Google Drive mounted")

In [ ]:
# Install required packages
!pip install -q google-auth google-auth-oauthlib google-auth-httplib2
!pip install -q google-api-python-client
print("✅ Dependencies installed")

In [ ]:
import os
import json
import time
import traceback
from datetime import datetime
from pathlib import Path
from google.oauth2 import service_account
from googleapiclient.discovery import build

class ColabProcessor:
    """Process Claude Tools requests in Colab"""
    
    def __init__(self, folder_id):
        self.folder_id = folder_id
        self.drive_service = None
        self.processed_requests = set()
        
    def initialize(self):
        """Initialize with user's Google Drive"""
        # Use the mounted drive's credentials
        from google.colab import auth
        auth.authenticate_user()
        
        # Build service
        self.drive_service = build('drive', 'v3')
        print(f"✅ Initialized with folder: {self.folder_id}")
        
    def list_requests(self):
        """List pending requests"""
        query = f"'{self.folder_id}' in parents and name contains 'request_' and trashed=false"
        results = self.drive_service.files().list(
            q=query,
            fields="files(id, name)"
        ).execute()
        
        requests = []
        for file in results.get('files', []):
            if file['name'].endswith('.json') and file['id'] not in self.processed_requests:
                requests.append(file)
        
        return requests
    
    def read_request(self, file_id):
        """Read request data"""
        content = self.drive_service.files().get_media(fileId=file_id).execute()
        return json.loads(content.decode('utf-8'))
    
    def write_response(self, request_id, response_data):
        """Write response back to Drive"""
        response_name = f"response_{request_id}.json"
        
        # Create response file
        file_metadata = {
            'name': response_name,
            'parents': [self.folder_id]
        }
        
        from googleapiclient.http import MediaIoBaseUpload
        import io
        
        media = MediaIoBaseUpload(
            io.BytesIO(json.dumps(response_data, indent=2).encode('utf-8')),
            mimetype='application/json'
        )
        
        self.drive_service.files().create(
            body=file_metadata,
            media_body=media
        ).execute()
        
        print(f"✅ Response written: {response_name}")
    
    def execute_code(self, code):
        """Execute code and capture output"""
        from io import StringIO
        import sys
        
        old_stdout = sys.stdout
        sys.stdout = StringIO()
        
        try:
            exec(code, {'__name__': '__main__'})
            output = sys.stdout.getvalue()
            return {
                'status': 'success',
                'output': output,
                'timestamp': time.time()
            }
        except Exception as e:
            return {
                'status': 'error',
                'error': str(e),
                'traceback': traceback.format_exc(),
                'timestamp': time.time()
            }
        finally:
            sys.stdout = old_stdout
    
    def process_request(self, request_file):
        """Process a single request"""
        request_id = request_file['name'].replace('request_', '').replace('.json', '')
        print(f"\n📋 Processing request: {request_id}")
        
        try:
            # Read request
            request_data = self.read_request(request_file['id'])
            print(f"   Type: {request_data.get('type')}")
            
            # Process based on type
            if request_data['type'] == 'execute_code':
                result = self.execute_code(request_data['code'])
            else:
                result = {
                    'status': 'error',
                    'error': f"Unknown request type: {request_data['type']}"
                }
            
            # Write response
            self.write_response(request_id, result)
            
            # Mark as processed
            self.processed_requests.add(request_file['id'])
            
        except Exception as e:
            print(f"❌ Error processing request: {e}")
            self.write_response(request_id, {
                'status': 'error',
                'error': str(e),
                'timestamp': time.time()
            })
    
    def run(self, poll_interval=5):
        """Run the processor loop"""
        print(f"🚀 Starting Colab Processor")
        print(f"📁 Monitoring folder: {self.folder_id}")
        print(f"⏱️  Poll interval: {poll_interval}s")
        print("Press Ctrl+C to stop\n")
        
        try:
            while True:
                # Check for new requests
                requests = self.list_requests()
                
                if requests:
                    print(f"\n📨 Found {len(requests)} new request(s)")
                    for req in requests:
                        self.process_request(req)
                else:
                    print(f"⏳ No new requests... ({datetime.now().strftime('%H:%M:%S')})", end='\r')
                
                time.sleep(poll_interval)
                
        except KeyboardInterrupt:
            print("\n\n🛑 Processor stopped")

# Configuration
FOLDER_ID = "1tzHn4J3QntSLJlJNXcNJe3cLdILGEb3Z"  # Your Drive folder

# Initialize and run
processor = ColabProcessor(FOLDER_ID)
processor.initialize()

In [ ]:
# Start processing requests
# This will run continuously until you stop it
processor.run(poll_interval=3)

## Test the Processor
Run this cell to test if everything is working:

In [ ]:
# Quick test
test_code = """
print("Hello from Colab!")
print(f"Working directory: {os.getcwd()}")

# Check environment
import platform
print(f"Python: {platform.python_version()}")
print(f"System: {platform.system()} {platform.release()}")

# Check GPU
try:
    import torch
    if torch.cuda.is_available():
        print(f"GPU: {torch.cuda.get_device_name(0)}")
    else:
        print("No GPU available")
except ImportError:
    print("PyTorch not installed")
"""

result = processor.execute_code(test_code)
print("Test Result:")
print(result['output'] if result['status'] == 'success' else result['error'])